# Estudo do caso $d_A = 2$ e $d_B = 3$

In [1]:
import functions as fc
import numpy as np
import picos as pic

In [4]:
# Creating the SDP_LHS(m,k,rho,rho_sep,eta,detp,medicoes) function
def SDP_LHS_qubit_trit(m,k,rho,rho_sep,eta,detp,medicoes):

    #Creating the problem
    P = pic.Problem()

    #Creating the optimization variables
    q = pic.RealVariable('q')

    chi = pic.HermitianVariable('chi',(6,6))

    sigma = [pic.HermitianVariable('Sigma_lambda[{}]'.format(i),(3,3)) for i in range(k**m)]

    rho_q = rho*q+(1-q)*rho_sep

    rho_eta = eta*chi+(1-eta)*(pic.partial_trace(rho_sep,subsystems=1,dimensions=(2,2)))@(pic.partial_trace(chi,subsystems=0,dimensions=(3,3)))

    est_det = [pic.sum([sigma[j]*detp[j,i] for j in range(k**m)]) for i in range(k*m)]

    est = [(np.kron(medicoes[i],np.eye(3)))*chi for i in range(k*m)]

    #Creating the constraints
    P.add_constraint(q<=1)

    P.add_constraint(q>=0)

    P.add_list_of_constraints([sigma[i]>>0 for i in range(k**m)]) 

    P.add_constraint(rho_q == rho_eta)

    P.add_list_of_constraints([pic.partial_trace(est[i],subsystems=0,dimensions=(3,3))==est_det[i] for i in range(k*m)])

    #Setting the objective
    P.set_objective('max',q)

    #Finding the solution
    solution = P.solve()

    #Return the problem created, the solution found, the value of q
    return P, solution, q

In [8]:
rho_sep = np.eye(6)/6

rho = fc.rho_mixed_HS(6)

medicoes, eta = fc.measurements(1,PLOT=False)

m = int(medicoes.shape[0]/2)

detp = fc.strategies_LHS(m,2)

P, solution, q = SDP_LHS_qubit_trit(m,2,rho,rho_sep,eta,detp,medicoes)

print(solution)

TypeError: Subsystem dimensions do not match expression.